In [1]:
from datasets import load_dataset

# Load the dataset from the JSONL file
dataset = load_dataset('json', data_files='user_queries.jsonl')

# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

df.head()

,conversation_hash,first_query
0,34e6bc4f50a9cd0c5a5f947ddbb9824b,"Hi there, Can you help me revise my love story..."
1,e1dda5bd84cb41a924308748c112243c,[VARIABLE] = I want a very fast epic final bos...
2,7f780423584ccd13562f838ca9496b1d,"You, ChatGPT, will be my prompt engineer. We w..."
3,d9dbe9e0356aa594d9d20adb65f7ed3a,What is the derivative of ln((x+a)/(x+b)) with...
4,fb9210f01518ec83a7203ea51a23fc4c,can you predict the next thing in the followin...


In [10]:
import os
from dotenv import load_dotenv
import requests
import json
from tqdm import tqdm

load_dotenv()

# Get current number of lines in reasoning-messages.jsonl
try:
    with open('reasoning-messages.jsonl', 'r') as f:
        num_lines = sum(1 for line in f)
    print(f"Current number of lines in reasoning-messages.jsonl: {num_lines}")
except FileNotFoundError:
    print("reasoning-messages.jsonl does not exist yet")
    num_lines = 0

# Iterate through each query in the DataFrame
remaining = df.iloc[num_lines:]
for index, row in tqdm(remaining.iterrows(), total=len(remaining)):
    query = row['first_query']

    url = "https://api.fireworks.ai/inference/v1/chat/completions"
    payload = {
    "model": "accounts/fireworks/models/deepseek-r1",
    "max_tokens": 20480,
    "top_p": 1,
    "top_k": 40,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "temperature": 0.6,
    "messages": [
        {
        "role": "user",
        "content": query
        }
    ]
    }
    headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.getenv("FIREWORKS_API_KEY")
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
    res = response.json()['choices'][0]['message']['content']
    output = [
        {
            "role": "user",
            "content": query
        },
        {
            "role": "assistant",
            "content": res
        }
    ]
    with open('reasoning-messages.jsonl', 'a') as f:
        json.dump({"messages": output}, f)
        f.write('\n')

# Start: $271.70
# End: $

Current number of lines in reasoning-messages.jsonl: 84


100%|██████████| 16/16 [08:05<00:00, 30.32s/it]
